In [1]:
import pandas as pd
import urllib.request
import shutil
import os
import pathlib

%load_ext autoreload
%autoreload 2
from utils import *

In [2]:
processed_subjects = os.listdir("videos/keypoints")

In [3]:
videos = pd.read_csv("videos.tsv", sep='\t',header=None)
videos = videos.drop([0,1,2,5], axis=1)
videos.columns =['ID', 'path', ] 

In [4]:
videos = videos[~videos.ID.isin(processed_subjects)]
videos

,ID,path


In [5]:
# Download videos
root_url = "https://motionlab-storage.s3.amazonaws.com/media/"

for index, row in videos.iterrows():
    filename, file_extension = os.path.splitext(row[1])
    url = root_url + row[1]
    path = "videos/" + row[0] + file_extension

    # Download the file from `url` and save it locally under `file_name`:
    with urllib.request.urlopen(url) as response, open(path, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

In [6]:
# Run openpose
dir_path = pathlib.Path().absolute()
for index, row in videos.iterrows():
    filename, file_extension = os.path.splitext(row[1])
    path = row[0] + file_extension
    os.makedirs("videos/keypoints/" + row[0], exist_ok=True)
    cmd = "docker run --gpus=1 -v {}/videos:/openpose/data stanfordnmbl/openpose-gpu\
    /openpose/build/examples/openpose/openpose.bin --video /openpose/data/{}\
    --display 0 --write_json /openpose/data/keypoints/{} --render_pose 0".format(dir_path, path, row[0])
    print(cmd)
    os.system(cmd)

In [7]:
# Convert to numpy arrays
os.makedirs("videos/np", exist_ok=True)

for index, row in videos.iterrows():
    subjectid = row[0]
    res = json2np("videos/keypoints/{}".format(subjectid),subjectid)
    np.save("videos/np/{}.npy".format(subjectid), res)